# **Аннотация генами геномов de novo**
# GLIMMER
Загрузите

In [ ]:
! wget http://ccb.jhu.edu/software/glimmer/glimmer302b.tar.gz

Выполните последовательно следующие команды (объясните их смысл)


In [ ]:
%%bash
tar xzf glimmer302b.tar.gz
cd glimmer3.02/
cd src/
make

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Необходимые нам тестовые варианты находятся в папке ./glimmer3.02/sample-run

Это каталог, содержащий образец прогона Glimmer3. В нем содержится
последовательность генома Treponema pallidum (файл tpall.fna) и
список аннотированных генов для него (файл tpall.nh), оба загружены из GenBank.

Этапы: Сперва нам необходимо натренировать модель. Для этого в папке
./glimmer3.02/sample-run уже есть пример с fasta (from-training.train)

**###---Запустим генерацию модели---###**

In [ ]:
!mkdir -p /content/glimmer3.02/sample-run_new

In [ ]:
! chmod 755 /content/glimmer3.02/sample-run/tpall.fna

Используем утилиту long-orfs

bin/long-orfs --help

USAGE: long-orfs [options] \<sequence-file> \<output-file>

In [ ]:
! /content/glimmer3.02/bin/long-orfs -n -t 1.15 /content/glimmer3.02/sample-run/tpall.fna /content/glimmer3.02/sample-run_new/orf.positions

Starting at Tue Jan 19 20:42:12 2021

Sequence file = /content/glimmer3.02/sample-run/tpall.fna
Excluded regions file = none
Circular genome = true
Initial minimum gene length = 90 bp
Determine optimal min gene length to maximize number of genes
Maximum overlap bases = 30
Start codons = atg,gtg,ttg
Stop codons = taa,tag,tga
Sequence length = 1138011
Final minimum gene length = 631
Number of genes = 514
Total bases = 674580


Найденные на предыдущем этапе ORF для тренировки - это результат работы готовой
модели. Мы воспользовались уже существующей моделью, только для того, чтобы
сгенерировать для себя обучающую выборку.
### Трансформируем отобранные позиции orf в fasta
Используем утилиту extract

In [ ]:
! /content/glimmer3.02/bin/extract -t /content/glimmer3.02/sample-run/tpall.fna /content/glimmer3.02/sample-run_new/orf.positions > /content/glimmer3.02/sample-run_new/orf.fasta

In [ ]:
!head /content/glimmer3.02/sample-run_new/orf.fasta

>00001  4 1398  len=1392
ATGGACGCAGTAGGGTATGAAGTATTCTGGAACGAGACACTCAGCCAGATACGGAGTGAA
TCGACCGAAGCAGAATTTAACATGTGGTTTGCTCATTTGTTCTTTATCGCATCTTTTGAA
AACGCTATCGAAATAGCAGTACCTTCAGACTTTTTCCGAATACAGTTTAGCCAAAAATAT
CAAGAAAAGCTTGAGCGCAAGTTCCTCGAACTTTCTGGACACCCCATTAAACTTTTGTTT
GCCGTTAAAAAAGGCACCCCTCATGGAAATACTGCTCCCCCCAAACACGTGCATACCTAC
CTGGAGAAAAACTCTCCTGCAGAGGTTCCTTCCAAAAAGAGCTTTCACCCCGACCTGAAC
AGAGACTATACCTTCGAGAACTTTGTATCCGGAGAAGAAACCAAATTCAGCCATAGCGCT
GCTATCTCCGTATCAAAAAACCCAGGCACTTCCTACAATCCGTTACTTATCTACGGTGGA
GTGGGACTAGGAAAAACCCACCTTATGCAGGCTATTGGACACGAGATCTACAAGACAACA


###---Теперь обучим марковскую модель---###
USAGE: build-icm [options] output_file < input-file

In [ ]:
! /content/glimmer3.02/bin/build-icm -r /content/glimmer3.02/sample-run_new/run1.icm < /content/glimmer3.02/sample-run_new/orf.fasta

###---Используем обученную модель, чтобы предсказать гены в геноме---###
USAGE: glimmer3 [options] \<sequence-file> \<icm-file> \<tag>

In [ ]:
! /content/glimmer3.02/bin/glimmer3 -o50 -g110 -t30 /content/glimmer3.02/sample-run/tpall.fna /content/glimmer3.02/sample-run_new/run1.icm /content/glimmer3.02/sample-run_new/run1

In [ ]:
! head /content/glimmer3.02/sample-run_new/run1.detail

Command:  /content/glimmer3.02/bin/glimmer3 -o 50 -g 110 -t 30 /content/glimmer3.02/sample-run/tpall.fna /content/glimmer3.02/sample-run_new/run1.icm /content/glimmer3.02/sample-run_new/run1

Sequence file = /content/glimmer3.02/sample-run/tpall.fna
Number of sequences = 1
ICM model file = /content/glimmer3.02/sample-run_new/run1.icm
Excluded regions file = none
List of orfs file = none
Input is NOT separate orfs
Independent (non-coding) scores are used
Circular genome = true


In [ ]:
! head /content/glimmer3.02/sample-run_new/run1.predict

>gi|15638995|ref|NC_000919.1| Treponema pallidum subsp. pallidum str. Nichols, complete genome
orf00001        4     1398  +1     6.22
orf00003     1641     2756  +3     2.89
orf00004     2776     3834  +1     5.47
orf00005     3863     4264  +2     2.77
orf00006     4391     6832  +2     7.08
orf00007     6832     7074  +1     0.25
orf00008     7317     7967  +3     6.92
orf00009     7997     8260  +2     2.91
orf00010     9515     8340  -3     2.80


Ваши результаты находятся в двух файлах

run1.predict

run1.detail

Изучите координаты полученных ORF. Используйте
/home/dmitry/soft/glimmer3.02/bin/extract, чтобы найти их fasta последовательность

Как бы вы нашли соответствуют ли данные последовательности генам?

Подсказка: Используйте BLAST, BLAT. Посмотрите на координаты этих последовательностей в геномном браузере.

EXTRA:

GeneMark

http://opal.biology.gatech.edu/GeneMark/

Prodigal

https://github.com/hyattpd/Prodigal